# This notebook parses and combines the datasets

In [765]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateparser
import datetime
import os
import json
import re
import difflib
import string

can_province_names = {
      'Alberta':'AB',
      'British Columbia':'BC',
      'Manitoba':'MB',
      'New Brunswick':'NB',
      'Newfoundland and Labrador':'NL',
      'Newfoundland':'NL',
      'Nova Scotia':'NS',
      'Northwest Territories':'NT',
      'Nunavut':'NU',
      'Ontario':'ON',
      'Prince Edward Island':'PE',
      'Quebec':'QC',
      'Saskatchewan':'SK',
      'Yukon':'YT',
    }


can_province_keys = {
      'AB':'Alberta',
      'BC':'British Columbia',
      'MB':'Manitoba',
      'NB':'New Brunswick',
      'NL':'Newfoundland and Labrador',
      'NS':'Nova Scotia',
      'NT':'Northwest Territories',
      'NU':'Nunavut',
      'ON':'Ontario',
      'PE':'Prince Edward Island',
      'QC':'Quebec',
      'SK':'Saskatchewan',
      'YT':'Yukon',
    }


def Assign_Death_Code(df,Keys,Cols,Type,Recat=999,Validate = False,ValCols=None):
    for k in Keys:
        for c in Cols:
            if Validate == True:
                print(df.loc[((df['Death_Category']==Recat)&
                    (df[c].str.contains(k, flags=re.IGNORECASE, regex=True))),
                   ValCols].values)
            else:
                df.loc[((df['Death_Category']==Recat)&
                    (df[c].str.contains(k, flags=re.IGNORECASE, regex=True))),
                   'Death_Category']=Death_Catageory_Code[Type]
            
def FindDates(s):
    flag = 'Parsed'
    if '(' in s and ')' in s:
        d = re.search(r'\((.*?)\)',s).group(1)
        date = dateparser.parse(d)
        if date is None:
            try:
                sp = d.split(',')
                sp1 = sp[-1]
                sp0 = sp[0].split(',')[-1]
                d = sp0.split('-')[-1].split('/')[-1]+sp1#[-1]
#                 d = sp[0].split('-')[-2].split('/')[0]+sp[-1]
                date = dateparser.parse(d)
            except:
                dates = datefinder.find_dates(s)
                D = []
                for d in dates:
                    D.append(d)
                if len(D) > 0:
                    date = D[-1]
                    flag = 'Found'
                else:
                    date = None
                    flag = 'Missing'
                pass
    else:
        dates = datefinder.find_dates(s)
        D = []
        for d in dates:
            D.append(d)
        if len(D) > 0:
            date = D[-1]
            flag = 'Found'
        else:
            date = None
            flag = 'Missing'
    return(date,flag)
            
Death_Catageory_Code = {
    'Shooting':1,
#     'Rubber Bullet':2,
    'Taser':3,
    'Other Weapon':4,
    'Excited Delirium':9,
    'Use of Force':10,
    'Accidental':15,
    'Overdose/Intoxication':25,
    'Vehicle':50,
    'In Custody':75,
    'Suicide':100,
    'Other/Unknown':999
}
Death_Category_SearchKeys = {
    'Shooting':['shoot','shot ','gunshot','rubber bullet','fired'],
#     'Rubber Bullet':[],
    'Taser':['tase','taze','stun gun'],
    'Other Weapon':['baton','club','pepper spray','pepper-spray'],
    'Excited Delirium':['excited delirium'],    
    'Use of Force':['Physical force','restrain','subdue','struggling','struggled','resisted','altercation',
                    'broken rib','collapsed lung',"beat and kick",'dragged','held him down'
#                     'handcuff','apprehended',
#                     'arrested','Anoxic brain injury','interact','trauma','bleeding','blood','strangling',
#                     'head injury','collapsed lung','pushed','fight','resisting','kicked','altercation',
#                     'respiratory failure','injuries'
                   ],
    'Accidental':['Fall','Fell','jump','hypothermia','drowning','suffocat'],
    'Overdose/Intoxication':['Methamphetamine','Cocaine','methadone','alcohol','pills','drugs',
                 'intoxication','overdose','toxicity','blocked with vomit'],
    'Vehicle':['crash','vehicle',"hit by officer's car",'collision'],
    'In Custody':['cell','custody','hospital','Died of an allergy','Found unresponsive','Segregation'],
    'Suicide':['suicide','suicidal','hang','hung','self-inflicted','Incisions of neck'],
    'Other/Unknown':['Unknown']
}

# CBC Deadly Force 2018

### This is the most detailed dataset.  It has:
* location
* race
* gender
* cause of death
* officer information

The data is in its original format, except I have edited a few charcaters/names for fomatting eg. J√¥van to Jovan

In [766]:
# incident.csv contains postal code of the incident, province, municipality, along with the date and incident ID
Incident = pd.read_csv(
    'DeadlyForce_2018/incident.csv',
    delimiter = ',',
    header = 0,
    parse_dates=['date'],
    index_col=['id_incident'], 
    encoding='utf-8'
)
# victim.csv contains information on the victms including age, race, etc.
Victim = pd.read_csv(
    'DeadlyForce_2018/victim.csv',
    delimiter = ',',
    header = 0, 
    index_col=['id_incident'], 
    encoding='utf-8'
)
#police.csv contaisn information about the police department and oficer involved
Police = pd.read_csv(
    'DeadlyForce_2018/police.csv',
    delimiter = ',',
    header = 0, 
    index_col=['id_incident'], 
    encoding='utf-8'
)
Join_1 = Incident.join(Victim)
Join_2 = Join_1.join(Police)
PID = Join_2.reset_index().set_index('date',drop = True)
# PID['CBC_2018_id'] = PID['id_incident'].astype(str)+'_CBC_2018'
PID['middle_name']=PID['middle_name'].fillna('')
PID['last_name']=PID['last_name'].fillna('')
PID['name'] = PID['first_name'] +' '+PID['middle_name']+' '+PID['last_name']
PID['name'] = PID['name'].str.replace('  ',' ')
PID['data_source'] = 'Deadly Force (2018)'
# PID['transgender'] = ''  # Only noted if explicitly indicated
# Chevranna was a trans woman killed by hamilton police, summary indicated she was "born male"
# PID.loc[PID['name'] == 'Chevranna Abdi','transgender'] = 'Transgender'
# print(PID.loc[PID['name'] == 'Chevranna Abdi'])
# print(PID['cause_death'].unique())
# print(PID['armed_type'].unique())
PID = PID.rename(columns={'Department':'department',
       'Charges':'charges','Officers Involved':'officers involved'})
# print(PID['substance_abuse'].unique())
# print(PID['mentral_distress_disorder'].unique())
# print(PID.Date)
PID = PID.fillna('')
PID['Check']='Yes'
PID.loc[PID['postal_code']!='','Check'] = 'No'
PID['ds_rank']=0
# print(PID.loc[PID['id_victim']=='0417-P1'])

# CBC Deadly Force 2020
### The updated datset is less detailed.


In [767]:
CBC = pd.read_csv('Deadly_Force_2020_Original.csv',
                        parse_dates=['DATE'],
#                         index_col=['VICTIM ID']
                        )
CBC = CBC.rename(columns={'AGE':'age','GENDER':'gender','RACE':'race','SUMMARY':'summary','PROV':'prov',
                          'NAME FIRST':'first_name','NAME MIDDLE':'middle_name','NAME LAST':'last_name',
                          'NICKNAME':'alias_nickname','CAUSE DEATH':'cause_death','ARMED TYPE':'armed_type',
                          'POLICE SERVICE':'department','DATE':'date','VICTIM ID':'id_victim'})
CBC = CBC.drop('PICTURE SOURCE',axis=1)
CBC = CBC.set_index('date')
CBC['middle_name']=CBC['middle_name'].fillna('')
CBC['last_name']=CBC['last_name'].fillna('')
CBC['name'] = CBC['first_name'] +' '+ CBC['middle_name'] +' '+ CBC['last_name']
CBC['name'] = CBC['name'].str.replace('  ',' ')
CBC['age']=CBC['age'].astype(float)
CBC = CBC.fillna('')
CBC['data_source'] = 'Deadly Force (2020)'
CBC['ds_rank']=1
# Update records since last 
for key in ['name','race','gender','cause_death','armed_type']:
    UK = PID.loc[((PID['id_victim'].isin(CBC['id_victim']))&(PID[key]=='Unknown')),'id_victim']
    Vals = CBC.loc[((CBC['id_victim'].isin(UK))&(CBC[key]!='Unknown'))]
    if Vals.shape[0]>0:
        print('Updates: ',key,Vals.shape[0])
        PID.loc[((PID['id_victim'].isin(Vals['id_victim']))&(PID[key]=='Unknown')),key]=Vals[key]
        PID.loc[((PID['id_victim'].isin(Vals['id_victim']))&(PID[key]=='Unknown')),'summary']+='; '+Vals['summary']
Missing = CBC.loc[CBC['id_victim'].isin(PID['id_victim'])==False].sort_index()
# print(Missing.columns)
PID = PID.append(Missing)

c = 0
for i, row in PID.iterrows():
    Temp = CBC.loc[((CBC['id_victim']==row['id_victim'])&((CBC['summary']!=row['summary'])))]
    if Temp.shape[0]>0:
        c += 1
        Sum = PID.loc[PID['id_victim']==row['id_victim'],'summary'].values
#         DS = PID.loc[PID['id_victim']==row['id_victim'],'data_source'].values
        try:
            PID.loc[PID['id_victim']==row['id_victim'],'summary'] = Sum +'; '+Temp['summary']
#             PID.loc[PID['id_victim']==row['id_victim'],'data_source'] = DS +'; '+Temp['data_source']
        except:
            pass
#         print(Temp)
# print(c)
# Two records with names are missing summaries
# https://www.baytoday.ca/local-news/no-charges-after-local-man-dies-during-arrest-1393616
PID.loc[PID['name']=='Gordon Dale Couvrette','summary'] = '''No charges will be laid against North Bay police
 officers after a 43-year-old Harris Drive man died after being tasered in his bedroom during an arrest last
 year.'''
# https://www.cbc.ca/news/canada/thunder-bay/police-shooting-lac-seul-first-nation-1.3579724
PID.loc[PID['name']=='Brian Gray','summary'] =  '''The jury at the inquest found that Gray died of\
'gunshot wounds to the torso.' It ruled the death homicide, which in a coroner's inquest means a death as a\
esult of the purposeful actions of another person and does not carry any criminal connotations.'''

# https://toronto.ctvnews.ca/siu-clears-cop-who-shot-and-killed-mississauga-teen-robbery-suspect-1.4160962
PID.loc[PID['name']=='Ozama Shaw','summary'] =  '''The province’s Special Investigations Unit (SIU) has cleared\
a Peel Regional Police officer who shot and killed 15-year-old Ozama Shaw in a Mississauga plaza\
last year, saying the incident appeared on its face to be a case of “shoot or be shot.”'''


# print(PID.loc[PID['summary']=='',['name','department','race','prov','summary','age']])
# print(PID.index)

print(PID.loc[PID['last_name']=='Shaw',['data_source','summary']].values)
PID = PID.replace({'Unknown':'Unspecified',
                   'Unknown ':'Unspecified'})
PID.loc[PID['name']=='Unspecified',['last_name','middle_name','first_name']]='Unspecified'
# print(PID['age'])
PID['age']=PID.age.replace({'':np.nan})
PID['age']=PID['age'].astype(float)
print(CBC.loc[CBC.id_victim=='0536-V1'])
PID.loc[PID['Check']!='No','Check']=='Yes'

Updates:  race 1
Updates:  cause_death 1
Updates:  armed_type 2
[['Deadly Force (2018)'
  'The province’s Special Investigations Unit (SIU) has cleareda Peel Regional Police officer who shot and killed 15-year-old Ozama Shaw in a Mississauga plazalast year, saying the incident appeared on its face to be a case of “shoot or be shot.”']]
           id_victim first_name last_name middle_name alias_nickname age  \
date                                                                       
2020-02-26   0536-V1    Unknown                                            

           gender race prov department cause_death armed_type  \
date                                                            
2020-02-26   Male        NU       RCMP     Gunshot    Unknown   

                                                      summary      name  \
date                                                                      
2020-02-26  Officers shot and killed a man, according to A...  Unknown    

           

date
2010-05-09    False
2018-01-27    False
2018-02-03    False
2018-02-22    False
2018-02-24    False
              ...  
2020-06-09    False
2020-06-12    False
2020-06-20    False
2020-06-20    False
2020-06-22    False
Name: Check, Length: 97, dtype: bool

# BC Specific Dataset

https://docs.google.com/spreadsheets/d/1aLNSF4Hkk9XdVKeuVU6ZrRO6GtSxT4t8TiMiQ6ptLrY/edit#gid=0

* Data has been pre-processed to remove typos
* I edited/fixed two repeated records maunally

2014.06.15 Jacob George Setah

2014.02.13	Gregory Douglas Lloyd

In [768]:
# Read data and parse dates
GS = pd.read_csv('BC_List_Edited.csv',parse_dates=['Date'])
GS['Circumstances'] = GS[['Circumstances',
                        'Coroners cause of death',
                        'Coroners type of death']].fillna('Unknown')
GS = GS.rename(columns={'Name':'name','Age':'age','Sex':'gender','Circumstances':'summary',
                        'Coroners cause of death':'cause_death','City':'city_town',
                          'Agency responsible':'department','Date':'date'})
GS['department'] = GS['department'].str.replace(' / ','; ')
GS['name'] = GS['name'].str.replace('TBD','Unspecified')
GS = GS.set_index('date')

Temp=GS['name'].str.split(' ',expand=True)#.str.len()
Temp['Length'] = GS['name'].str.split(' ').str.len()
# print(Temp[''])
GS['first_name']=Temp[0]
GS['middle_name']='Unspecified'
GS['last_name']='Unspecified'
GS['prov']='BC'
GS['id_victim']=[str(i)+'_GS' for i in range(GS.shape[0])]
GS['data_source'] = 'Georgia St (2017)'
for i in range(2,Temp['Length'].max()+1):
#     print(Temp.loc[Temp['Length']==i,[i-1]].shape)
    GS.loc[Temp['Length']==i,'last_name']=Temp.loc[Temp['Length']==i,[i-1]].values
#     print(GS.loc[Temp['Length']==i,'last_name'].shape)
for i in range(3,Temp['Length'].max()+1):
    GS.loc[Temp['Length']==i-1,'middle_name']=Temp.loc[Temp['Length']==i-1,[i-2]].values
# print(GS.columns)
GS = GS.fillna('')
GS['summary']=GS['summary']+'; '+GS['Additional notes']+'; '+GS['Coroners type of death']+\
'; '+GS['Coroners type of death']+'; '+GS['Mental illness / addiction ']
GS = GS.drop(['Additional notes','Coroners type of death','Coroners type of death','Mental illness / addiction '],axis=1)
# print(GS['Additional notes'])
GS['age']=GS.age.replace({'':np.nan})
GS['age']=GS['age'].astype(float)

PID_BU = PID.copy()
GS_BU = GS.copy()
GS['ds_rank']=2


In [769]:
# Check if there are any "same" last names within a 60 day window, allowing for typos.
# Check for missing names within a shorter widow, didnt find any tho
# Add any othere records
# PID = PID_BU.copy()
Window_d = 5
Thresh = .85
# GS = GS_BU.loc[((GS_BU['id_victim']=='155_GS')|(GS_BU['id_victim']=='154_GS'))]
GS['GS_match'] = 'None'
PID['GS_match'] = 'None'
BC = PID.loc[PID['prov']=='BC'].copy()
NoBC = PID.loc[PID['prov']!='BC'].copy()
for i,row in GS.iterrows():#.loc[GS['id_victim']=='56_GS']
    Subset = BC.loc[(
        ((np.abs(BC.index-i)<datetime.timedelta(Window_d))&
           (BC['city_town']==row['city_town']))|
        (np.abs(BC.index-i)<=datetime.timedelta(1))#&
#            (BC['city_town']==row['city_town']))
                    )]
#     if row['last_name']=='Wrigth':
#         print(row)
    if Subset.shape[0]>0:# and row['last_name']=='Wright':
#         print(Subset.shape)
#         if row['last_name']
        if row['last_name']!='Unspecified':
            Match1=difflib.get_close_matches(row['last_name'],Subset['last_name'], cutoff=Thresh)
#             print(row['last_name'],Subset['last_name'])
            if len(Match1)>1:
                Match2 = difflib.get_close_matches(row['first_name'],Subset.loc[Subset['last_name']==Match1[0],
                                                                                'first_name'], cutoff=Thresh-.2)
                S = Subset.loc[((Subset['last_name']==Match1[0])&(Subset['first_name']==Match2[0])),'id_victim'].values[0]
                PID.loc[PID['id_victim']==S,'GS_match'] = row['id_victim']
                GS.loc[GS['id_victim']==row['id_victim'],'GS_match']=Subset.loc[((Subset['last_name']==Match1[0])&(Subset['first_name']==Match2[0])),
                                                                                'id_victim'].values[0]
            if len(Match1)==1:
                
                PID.loc[PID['id_victim']==Subset.loc[Subset['last_name']==Match1[0],'id_victim'].values[0],
                        'GS_match'] = row['id_victim']
                GS.loc[GS['id_victim']==row['id_victim'],'GS_match']=Subset.loc[Subset['last_name']==Match1[0],
                                                                                'id_victim'].values[0]
        else:
            if Subset.loc[Subset['prov']==row['prov']].shape[0]==1:
                PID.loc[PID['id_victim']==Subset.loc[Subset['prov']==row['prov'],'id_victim'].values[0],'GS_match'] = row['id_victim']
                GS.loc[GS['id_victim']==row['id_victim'],'GS_match']=Subset.loc[Subset['prov']==row['prov'],'id_victim'].values[0]
            else:   
#                 print('XXXXX')
                print(row)
                print(Subset)
print(GS.shape[0])
print(PID.shape[0])
PID = PID.append(GS)
print(PID.shape[0])
PID2 = PID.copy()
for i,row in PID.loc[((PID['GS_match']!='None'))].iterrows():
    Match = PID.loc[PID['id_victim']==row['GS_match']]
    if row['id_victim'].split('_')[-1]!='GS':
        Match = PID.loc[PID['id_victim']==row['GS_match']]
        Match1=difflib.get_close_matches(row['last_name'],Match['last_name'], cutoff=Thresh)
#         print(Match1)
        if len(Match1)>0:
            PID.loc[PID['id_victim']==row['id_victim'],'summary'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],'summary'].iloc[0]
            if Match['city_town'][0]!=row['city_town']:
                PID.loc[PID['id_victim']==row['id_victim'],'Check'] = 'Yes'
                PID.loc[PID['id_victim']==row['id_victim'],
                        'city_town'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],
                                                          'city_town'].iloc[0]
            PID2 = PID2.loc[PID2['id_victim']!=Match['id_victim'].values[0]]
        elif i - Match.index<=datetime.timedelta(1):
            PID.loc[PID['id_victim']==row['id_victim'],'summary'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],'summary'].iloc[0]
            if Match['city_town'][0]!=row['city_town']:
                PID.loc[PID['id_victim']==row['id_victim'],'Check'] = 'Yes'
                PID.loc[PID['id_victim']==row['id_victim'],
                        'city_town'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],
                                                          'city_town'].iloc[0]
            PID2 = PID2.loc[PID2['id_victim']!=Match['id_victim'].values[0]]
        else:
            print(i - Match.index)
print(PID2.shape[0])
PID = PID2.copy()



174
559
733
672


In [770]:
# PID = PID.sort_values(by=['ds_rank','last_name'])
# # print(PID.shape)
PID=PID.loc[((PID['name']!='Unspecified')&(PID['name'].duplicated(keep='first')==False))]
# print(PID.loc[((PID['name']!='Unspecified')&#(PID['last_name']=='Wright')&
#                (PID['name'].duplicated(keep=False))),['name','last_name','id_victim','GS_match','city_town']])
# # print(PID.shape)

In [771]:
# print(PID)

### laCRAP is a datasest in French 
* It only lists names, ages, dates, and "where" by province, with a few major cities (montreal,toronto,vancouver,ottawa,edmonton) separated out *sometimes
* I've coppied pasted to a text file from this source: https://lacrap.org/liste-des-noms-des-personnes-decedees-aux-mains-de-la-police-au-canada
* I've change the formatting a bit so things worked, and fixed a few typos

In [772]:
Cities = ['Montreal','Montréal','Toronto','Ottawa','Vancouver','Edmonton']
Province = ['Quebec','Quebec','Ontario','Ontario','British Columbia','Alberta']
City_Prov = {k:v for k,v in zip(Cities,Province)}
# laCRAP['Prov']=laCRAP['Prov'].replace(City_Prove)
# City_Police = ['Service de police de la Ville de Montréal','Toronto Police Service','Ottawa Police Service',
#                'Vancouver Police Department','Edmonton Police Service']
# for City,Prov,pol in zip(Cities,Province,City_Police):
#     laCRAP.loc[laCRAP['Prov']==City,['Prov','City','POLICE SERVICE']]=[Prov,City,pol]


#  parses the pre-processed data, and reads the french dates using dateparser
laCRAP = pd.read_csv('laCRAP/Allnames.txt')
laCRAP = laCRAP.dropna().reset_index(drop=True)
laCRAP.Date = laCRAP.Date.apply(lambda x: dateparser.parse(x))
# This cell attributes the location information
Descriptors = (laCRAP.loc[laCRAP.age.str.len()>=5,'age'].str.split(' in ',n=1,expand=True))
laCRAP['prov'] = None
laCRAP['city_town'] = None
laCRAP['department'] = 'Unknown'
laCRAP['summary'] = ''
laCRAP['data_source'] = 'la CRAP (2021)'
laCRAP = laCRAP.rename(columns={'Date':'date'})

for i,l in Descriptors.iterrows():
    try:
        laCRAP.loc[laCRAP.index>i,'prov']=can_province_names[l[1]]
        laCRAP.loc[laCRAP.index>i,'cause_death']=l[0]
        laCRAP.loc[laCRAP.index>i,'city_town']='Unspecified'
    except:
        laCRAP.loc[laCRAP.index>i,'city_town']=l[1]
        laCRAP.loc[laCRAP.index>i,'prov']=can_province_names[City_Prov[l[1]]]
        laCRAP.loc[laCRAP.index>i,'summary']=l[0]
        
laCRAP = laCRAP.loc[laCRAP.date>'1987-01-01']
Temp=laCRAP['name'].str.split(' ',expand=True)
Temp['Length'] = laCRAP['name'].str.split(' ').str.len()
laCRAP['first_name']=Temp[0]
laCRAP['middle_name']=''
laCRAP['last_name']=''
laCRAP['id_victim']=[str(i)+'_CRAP' for i in range(laCRAP.shape[0])]
laCRAP['age']=laCRAP['age'].astype(float)#.fillna(999)

for i in range(2,Temp['Length'].max()+1):
    laCRAP.loc[Temp['Length']==i,'last_name']=Temp.loc[Temp['Length']==i,[i-1]].values
for i in range(3,Temp['Length'].max()+1):
    laCRAP.loc[Temp['Length']==i,'middle_name']=Temp.loc[Temp['Length']==i,[i-2]].values

laCRAP = laCRAP.set_index(pd.DatetimeIndex(laCRAP.date))
laCRAP = laCRAP.drop('date',axis=1)
laCRAP['ds_rank']=3
# print(laCRAP.loc[laCRAP['prov']=='MB'])
print(laCRAP)

                          name   age prov    city_town department  \
date                                                                
1987-04-16      Serge Laforest  33.0   QC     Montreal    Unknown   
1987-08-14          Mark White  30.0   QC     Montreal    Unknown   
1987-11-11     Anthony Griffin  19.0   QC     Montreal    Unknown   
1988-06-20    Bernard Laforest  26.0   QC     Montreal    Unknown   
1988-10-07  José Carlos Garcia  43.0   QC     Montreal    Unknown   
...                        ...   ...  ...          ...        ...   
2017-05-01      Jeremy Nuvviaq  39.0   NU  Unspecified    Unknown   
2020-02-26    Attachie Ashoona  38.0   NU  Unspecified    Unknown   
2020-05-05    Abraham Natanine  31.0   NU  Unspecified    Unknown   
2009-12-31    Adamie Nuturaluk  56.0   NU  Unspecified    Unknown   
2016-05-19         Adla Pudlat  29.0   NU  Unspecified    Unknown   

                                                      summary     data_source  \
date                 

In [773]:
# Check if there are any "same" last names within a 60 day window, allowing for typos.
# Check for missing names within a shorter widow, didnt find any tho
# Add any othere records
# PID = PID_BU.copy()
Window_d = 4
Thresh = .85
laCRAP['laCRAP_match'] = 'None'
PID['laCRAP_match'] = 'None'
print(PID.shape)
print(laCRAP.shape)
# BC = PID.loc[PID['prov']=='BC'].copy()
# NoBC = PID.loc[PID['prov']!='BC'].copy()
for i,row in laCRAP.iterrows():#.loc[laCRAP['id_victim']=='638_CRAP']
    Subset = PID.loc[((np.abs(PID.index-i)<datetime.timedelta(Window_d))&
                           (PID['prov']==row['prov']))]
    if Subset.shape[0]>0:# :
        if row['last_name']!='Unspecified':
            Match1=difflib.get_close_matches(row['last_name'],Subset['last_name'], cutoff=Thresh)
            if len(Match1)>1:
                Match2 = difflib.get_close_matches(row['first_name'],Subset.loc[Subset['last_name']==Match1[0],
                                                                                'first_name'], cutoff=Thresh-.2)
                print(Match1,Subset,Match2,row)
                S = Subset.loc[((Subset['last_name']==Match1[0])&(Subset['first_name']==Match2[0])),
                               'id_victim'].values[0]
                PID.loc[PID['id_victim']==S,'laCRAP_match'] = row['id_victim']
                laCRAP.loc[laCRAP['id_victim']==row['id_victim'],'laCRAP_match']=Subset.loc[((Subset['last_name']==Match1[0])&(Subset['first_name']==Match2[0])),
                                                                                'id_victim'].values[0]
            if len(Match1)==1:
#                 print(Subset['name'])
                PID.loc[PID['id_victim']==Subset.loc[Subset['last_name']==Match1[0],'id_victim'].values[0],
                        'laCRAP_match'] = row['id_victim']
                laCRAP.loc[laCRAP['id_victim']==row['id_victim'],'laCRAP_match']=Subset.loc[Subset['last_name']==Match1[0],
                                                                                'id_victim'].values[0]
#                 print(PID.loc[PID['id_victim']==Subset.loc[Subset['last_name']==Match1[0],'id_victim'].values[0],
#                         'laCRAP_match'])
        else:
            if Subset.loc[Subset['prov']==row['prov']].shape[0]==1:
                print(row['name'],Match['name'])
                PID.loc[PID['id_victim']==Subset.loc[Subset['prov']==row['prov'],'id_victim'].values[0],'laCRAP_match'] = row['id_victim']
                laCRAP.loc[laCRAP['id_victim']==row['id_victim'],'laCRAP_match']=Subset.loc[Subset['prov']==row['prov'],'id_victim'].values[0]
            else:   
                print(row)
                print(Subset)
# print(laCRAP['laCRAP_match'].sort_values())
# print(PID['laCRAP_match'].sort_values())
print(laCRAP.shape[0])
PID = PID.append(laCRAP)
print(PID.shape[0])
PID2 = PID.copy()
for i,row in PID.loc[((PID['laCRAP_match']!='None'))].iterrows():
    Match = PID.loc[PID['id_victim']==row['laCRAP_match']]
    if row['id_victim'].split('_')[-1]!='CRAP':
        Match = PID.loc[PID['id_victim']==row['laCRAP_match']]
        Match1=difflib.get_close_matches(row['last_name'],Match['last_name'], cutoff=Thresh)
        if len(Match1)>0:
            PID.loc[PID['id_victim']==row['id_victim'],'summary'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],'summary'].iloc[0]
            if Match['city_town'][0]!=row['city_town']:
                PID.loc[PID['id_victim']==row['id_victim'],'Check'] = 'Yes'
                PID.loc[PID['id_victim']==row['id_victim'],
                        'city_town'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],
                                                          'city_town'].iloc[0]
            PID2 = PID2.loc[PID2['id_victim']!=Match['id_victim'].values[0]]
        elif i - Match.index==datetime.timedelta(0):
            print(row['name'],Match['name'])
            PID.loc[PID['id_victim']==row['id_victim'],'summary'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],'summary'].iloc[0]
            if Match['city_town'][0]!=row['city_town']:
                PID.loc[PID['id_victim']==row['id_victim'],'Check'] = 'Yes'
                PID.loc[PID['id_victim']==row['id_victim'],
                        'city_town'] += ';    '+Match.loc[Match['id_victim']!=row['id_victim'],
                                                          'city_town'].iloc[0]
            PID2 = PID2.loc[PID2['id_victim']!=Match['id_victim'].values[0]]
print(PID.shape[0]-PID2.shape[0],PID2.shape[0])

PID = PID2.copy()

(597, 36)
(1275, 14)
['Williams', 'Williams']             id_incident day_week prov city_town address_intersection  \
date                                                                   
2016-04-21        112.0      Thu   BC  Granisle         Morrison St.   
2016-04-21        112.0      Thu   BC  Granisle         Morrison St.   

           postal_code location_type id_victim first_name middle_name  ...  \
date                                                                   ...   
2016-04-21     V0J 1W0         Rural   0112-V1    Shirley    Beatrice  ...   
2016-04-21     V0J 1W0         Rural   0112-V2      Jovan              ...   

              charge_type charges  officers involved  \
date                                                   
2016-04-21  Under Invest.     TBD            0112-P1   
2016-04-21  Under Invest.     TBD            0112-P1   

                                 name          data_source Check ds_rank  \
date                                               

In [774]:
PID = PID2.copy()

window_d=16
A=(PID.loc[((PID['name']!='Unspecified')&
               (PID['name'].duplicated(keep='last')==True)),#&(PID['name'].duplicated(keep='first')==True)),
              ['name','last_name','prov','ds_rank','laCRAP_match','id_victim','city_town']].sort_values('name'))

B = (PID.loc[((PID['name']!='Unspecified')&
               (PID['name'].duplicated(keep='first')==True)),#&(PID['name'].duplicated(keep='first')==True)),
              ['name','last_name','prov','ds_rank','laCRAP_match','id_victim','city_town']].sort_values('name'))
# print(A)
# print(B)
print(PID.shape)
# print(A.loc[A['prov']!=B['prov'].values])
# print(B.loc[B['prov']!=A['prov'].values])
# print(np.abs(B['prov'].index-A['prov'].index))
PID = PID.reset_index(drop=False).set_index('id_victim')
# print(B.loc[np.abs(B['prov'].index-A['prov'].index)<=datetime.timedelta(Window_d)])
PID = PID.drop(B.loc[np.abs(B['prov'].index-A['prov'].index)<=datetime.timedelta(Window_d),'id_victim'].values)
PID = PID.drop(B.loc[np.abs(B['prov'].index-A['prov'].index)==datetime.timedelta(365),'id_victim'].values)
PID = PID.drop(B.loc[np.abs(B['prov'].index-A['prov'].index)==datetime.timedelta(366),'id_victim'].values)
PID = PID.drop(B.loc[np.abs(B['prov'].index-A['prov'].index)==datetime.timedelta(364),'id_victim'].values)
print(PID.shape)

(1445, 36)
(1430, 36)


In [775]:
# print(PID.loc[((PID['name']!='Unspecified') & (PID['last_name'].duplicated(keep=False))),
#               ['last_name','id_victim','prov','laCRAP_match']].sort_values(by='last_name').shape)
PID['race'] = PID['race'].replace({'Arab':'Middle Eastern',
                                  'Caucasian':'White',
                                  '':'Unspecified'})
print(PID)
# print(PID.loc[PID['last_name']!='Unspecified'].duplicated(keep=False).sum())

                date  id_incident day_week prov    city_town  \
id_victim                                                      
0001-V1   2012-01-06          1.0      Fri   QC     Montréal   
0002-V1   2012-01-11          2.0      Wed   AB       Onoway   
0003-V1   2012-01-12          3.0      Thu   ON     Oakville   
0004-V1   2012-02-03          4.0      Fri   ON      Toronto   
0005-V1   2012-02-13          5.0      Mon   ON     Hamilton   
...              ...          ...      ...  ...          ...   
1269_CRAP 2017-03-18          NaN      NaN   NU  Unspecified   
1271_CRAP 2020-02-26          NaN      NaN   NU  Unspecified   
1272_CRAP 2020-05-05          NaN      NaN   NU  Unspecified   
1273_CRAP 2009-12-31          NaN      NaN   NU  Unspecified   
1274_CRAP 2016-05-19          NaN      NaN   NU  Unspecified   

                         address_intersection postal_code location_type  \
id_victim                                                                 
0001-V1          

In [776]:
PID['ds_rank'] = 999
for i,s in enumerate(['Deadly Force (2018)','Deadly Force (2020)','Georgia St (2017)','la CRAP (2021)']):
    PID.loc[PID['data_source']==s,'ds_rank']=i
PID = PID.sort_values(by='ds_rank')

# print(PID.loc[PID['ds_rank']==999])
                     
print(PID.shape)
Named = PID.loc[PID['name']!='Unspecified']
Unspecified = PID.loc[PID['name']=='Unspecified']
# # Named = PID.loc[PID['name']=='Unknown']
# print(PID.groupby('name').count()['id_victim'].sort_values().index)
# # print(PID.loc[PID['name']=='Unknown'])
print(Named.shape)
Named = Named.loc[Named['name'].duplicated(keep=False)==False]
print(Named.shape)

PID = Named.append(Unspecified)
print(PID.shape)

(1430, 36)
(1430, 36)
(1400, 36)
(1400, 36)


In [778]:

# PID.index=PID.index.rename('Date')
print(PID)
print(PID.head())
PID.to_csv('CombinedSourceData.csv')

                date  id_incident day_week prov    city_town  \
id_victim                                                      
0001-V1   2012-01-06          1.0      Fri   QC     Montréal   
0333-V1   2001-07-29        333.0      Sun   ON     Bradford   
0332-V1   2007-12-27        332.0      Thu   BC       Vernon   
0331-V1   2007-12-10        331.0      Mon   BC    Vancouver   
0330-V1   2007-11-19        330.0      Mon   BC   Chilliwack   
...              ...          ...      ...  ...          ...   
367_CRAP  2019-08-15          NaN      NaN   QC  Unspecified   
368_CRAP  2019-08-29          NaN      NaN   QC  Unspecified   
369_CRAP  2019-11-07          NaN      NaN   QC  Unspecified   
291_CRAP  1997-03-18          NaN      NaN   QC  Unspecified   
1274_CRAP 2016-05-19          NaN      NaN   NU  Unspecified   

                  address_intersection postal_code location_type  \
id_victim                                                          
0001-V1    1000 rue De La Gauch

In [763]:
print(laCRAP.loc[laCRAP['last_name']=='Ashoona'])
print(PID.loc[PID['id_victim']=='0536-V1'])

                        name   age prov    city_town department  \
date                                                              
2020-02-26  Attachie Ashoona  38.0   NU  Unspecified    Unknown   

                                                      summary     data_source  \
date                                                                            
2020-02-26  Persons who died following a traffic incident ...  la CRAP (2021)   

                                                 cause_death first_name  \
date                                                                      
2020-02-26  Persons who died following a police intervention   Attachie   

           middle_name last_name  id_victim  ds_rank laCRAP_match  
date                                                               
2020-02-26               Ashoona  1271_CRAP        3         None  


KeyError: 'id_victim'